# Integrar com o Drive


# Resgatar repositorio Git

---






In [ ]:
!git clone https://github.com/korouuuuu/HMA.git
%cd HMA

fatal: destination path 'HMA' already exists and is not an empty directory.
/content/HMA


# Instalar dependencias


In [ ]:
!pip install basicsr==1.4.2.0
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt
!python setup.py develop
!pip install gdown

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu113 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1.2+rocm5.6, 2.2.0, 2.2.0+cpu, 2.2.0+cpu.cxx11.abi, 2.2.0+cu118, 2.2.0+cu121, 2.2.0+rocm5.6, 2.2.

# Criar diretorios de entrada e saida


In [ ]:
import os

os.makedirs("inputs_imgs", exist_ok=True)
os.makedirs("outputs", exist_ok=True)
os.makedirs("experiments/pretrained_models", exist_ok=True)

# 📥 Etapa 3: Baixar modelo Real-DRCT-GAN_Finetuned (via gdown)

Baixar modelo pré treinado de sua escolha: [acesse aqui](https://drive.google.com/drive/folders/1QJHdSfo-0eFNb96i8qzMJAPw31u9qZ7U)

In [ ]:
# 📥 Etapa 3: Baixar arquivo pré-treinado do Google Drive (via gdown)
import gdown

# URL do arquivo do Google Drive
file_url = "https://drive.google.com/uc?id=1D-STrev2r4ouMoE-qAgPhdRB0Co9zmBe"
# Caminho onde o arquivo será salvo (diretório atual ./)
output_path = "./experiments/pretrained_models/HMA_SRx4_pretrain.pth"

# Baixar o arquivo
gdown.download(file_url, output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1D-STrev2r4ouMoE-qAgPhdRB0Co9zmBe
From (redirected): https://drive.google.com/uc?id=1D-STrev2r4ouMoE-qAgPhdRB0Co9zmBe&confirm=t&uuid=25244af9-2c3b-4f6a-92d1-d34a626f72a8
To: /content/HMA/experiments/pretrained_models/HMA_SRx4_pretrain.pth
100%|██████████| 561M/561M [00:18<00:00, 31.1MB/s]


'./experiments/pretrained_models/HMA_SRx4_pretrain.pth'


# 📥 Etapa 4: Baixar imagem de teste

In [ ]:
img_url = "https://drive.google.com/uc?id=1B4Qafq44qYE1G838DW4TnjirIztFBEbb"
gdown.download(img_url, "inputs_imgs/test.png", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1B4Qafq44qYE1G838DW4TnjirIztFBEbb
To: /content/HMA/inputs_imgs/test.png
100%|██████████| 71.3k/71.3k [00:00<00:00, 70.5MB/s]


'inputs_imgs/test.png'

# 📄 Cria o YAML de configuração

In [ ]:
# Criar arquivo YAML de configuração para o teste
yaml_content = """
# general settings
name: HMA_test_X4
model_type: HMAModel
scale: 4
num_gpu: auto
manual_seed: 1234

# dataset and data loader settings
datasets:
  test_1:
    name: SingleImageDataset  # Alterado para refletir o tipo de dataset único
    type: SingleImageDataset  # Tipo ajustado para lidar com uma única imagem
    dataroot_lq: ./inputs_imgs  # Caminho para a imagem única de baixa qualidade
    io_backend:
      type: disk

# network structures
network_g:
  type: HMANet
  upscale: 4
  in_chans: 3
  img_size: 64
  window_size: 16
  interval_size: 4
  img_range: 1.
  depths: [6, 6, 6, 6, 6, 6]
  embed_dim: 180
  num_heads: [6, 6, 6, 6, 6, 6]
  mlp_ratio: 2
  upsampler: 'pixelshuffle'
  resi_connection: '1conv'

# path
path:
  pretrain_network_g: ./experiments/pretrained_models/HMA_SRx4_pretrain.pth
  strict_load_g: true
  param_key_g: 'params_ema'

# validation settings
val:
  save_img: true
  suffix: ~  # add suffix to saved images, if None, use exp name

  metrics:
    psnr: # metric name, can be arbitrary
      type: calculate_psnr
      crop_border: 4
      test_y_channel: true
    ssim:
      type: calculate_ssim
      crop_border: 4
      test_y_channel: true

# tile settings
tile:
  tile_size: 256
  tile_pad: 32
"""

# Salvar o arquivo YAML
with open('options/test/HMA_SRx4.yml', 'w') as f:
    f.write(yaml_content)

In [ ]:
!sed -i '/def rgb2ycbcr(img, y_only=False):/,+10d' /content/HMA/hma/data/paired_imagenet_dataset.py

!echo -e "def rgb2ycbcr(img, y_only=False):\n    import numpy as np\n    img = img.astype(np.float32)\n    if y_only:\n        rlt = (img[..., 0] * 65.481 + img[..., 1] * 128.553 +\n               img[..., 2] * 24.966 + 16.0) / 255.0\n    else:\n        rlt = np.empty_like(img)\n        rlt[..., 0] = (img[..., 0] * 65.481 + img[..., 1] * 128.553 +\n                       img[..., 2] * 24.966 + 16)\n        rlt[..., 1] = (-img[..., 0] * 37.797 - img[..., 1] * 74.203 +\n                       img[..., 2] * 112.0 + 128)\n        rlt[..., 2] = (img[..., 0] * 112.0 - img[..., 1] * 93.786 -\n                       img[..., 2] * 18.214 + 128)\n        rlt = rlt / 255.0\n    return rlt" >> /content/HMA/hma/data/paired_imagenet_dataset.py

code = '''
import torch
from torch.utils import data
import cv2
import numpy as np
from basicsr.utils import FileClient, imfrombytes, img2tensor
from basicsr.utils.matlab_functions import imresize
from basicsr.utils.registry import DATASET_REGISTRY

@DATASET_REGISTRY.register()
class PairedImageNetDataset(data.Dataset):

    def __init__(self, opt):
        super(PairedImageNetDataset, self).__init__()
        self.opt = opt
        self.file_client = None
        self.io_backend_opt = opt['io_backend']
        self.mean = opt['mean'] if 'mean' in opt else None
        self.std = opt['std'] if 'std' in opt else None
        self.gt_folder = opt['dataroot_gt']

        if self.io_backend_opt['type'] == 'lmdb':
            self.io_backend_opt['db_paths'] = [self.gt_folder]
            self.io_backend_opt['client_keys'] = ['gt']
            self.paths = paths_from_lmdb(self.gt_folder)
        elif 'meta_info_file' in self.opt:
            with open(self.opt['meta_info_file'], 'r') as fin:
                self.paths = [osp.join(self.gt_folder, line.split(' ')[0]) for line in fin]
        else:
            self.paths = sorted(list(scandir(self.gt_folder, full_path=True)))

    def __getitem__(self, index):
        if self.file_client is None:
            self.file_client = FileClient(self.io_backend_opt.pop('type'), **self.io_backend_opt)

        scale = self.opt['scale']
        gt_path = self.paths[index]
        img_bytes = self.file_client.get(gt_path, 'gt')
        img_gt = imfrombytes(img_bytes, float32=True)
        size_h, size_w, _ = img_gt.shape
        size_h = size_h - size_h % scale
        size_w = size_w - size_w % scale
        img_gt = img_gt[0:size_h, 0:size_w, :]
        size_h = max(size_h, self.opt['gt_size'])
        size_w = max(size_w, self.opt['gt_size'])
        img_gt = cv2.resize(img_gt, (size_w, size_h))
        img_lq = imresize(img_gt, 1 / scale)
        img_gt = np.ascontiguousarray(img_gt, dtype=np.float32)
        img_lq = np.ascontiguousarray(img_lq, dtype=np.float32)

        if self.opt['phase'] == 'train':
            gt_size = self.opt['gt_size']
            img_gt, img_lq = paired_random_crop(img_gt, img_lq, gt_size, scale, gt_path)
            img_gt, img_lq = augment([img_gt, img_lq], self.opt['use_hflip'], self.opt['use_rot'])

        if 'color' in self.opt and self.opt['color'] == 'y':
            img_gt = rgb2ycbcr(img_gt, y_only=True)[..., None]
            img_lq = rgb2ycbcr(img_lq, y_only=True)[..., None]

        if self.opt['phase'] != 'train':
            img_gt = img_gt[0:img_lq.shape[0] * scale, 0:img_lq.shape[1] * scale, :]

        img_gt, img_lq = img2tensor([img_gt, img_lq], bgr2rgb=True, float32=True)
        if self.mean is not None or self.std is not None:
            normalize(img_lq, self.mean, self.std, inplace=True)
            normalize(img_gt, self.mean, self.std, inplace=True)

        return {'lq': img_lq, 'gt': img_gt, 'gt_path': gt_path}

    def __len__(self):
        return len(self.paths)

def rgb2ycbcr(img, y_only=False):
    import numpy as np
    img = img.astype(np.float32)
    if y_only:
        rlt = (img[..., 0] * 65.481 + img[..., 1] * 128.553 +
               img[..., 2] * 24.966 + 16.0) / 255.0
    else:
        rlt = np.empty_like(img)
        rlt[..., 0] = (img[..., 0] * 65.481 + img[..., 1] * 128.553 +
                       img[..., 2] * 24.966 + 16)
        rlt[..., 1] = (-img[..., 0] * 37.797 - img[..., 1] * 74.203 +
                       img[..., 2] * 112.0 + 128)
        rlt[..., 2] = (img[..., 0] * 112.0 - img[..., 1] * 93.786 -
                       img[..., 2] * 18.214 + 128)
        rlt = rlt / 255.0
    return rlt
'''

with open('/content/HMA/hma/data/paired_imagenet_dataset.py', 'w') as f:
    f.write(code)




# Rodar inferência


In [ ]:
!cat /content/HMA/hma/data/paired_imagenet_dataset.py
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py
!python hma/test.py -opt options/test/HMA_SRx4.yml


import torch
from torch.utils import data
import cv2
import numpy as np
from basicsr.utils import FileClient, imfrombytes, img2tensor
from basicsr.utils.matlab_functions import imresize
from basicsr.utils.registry import DATASET_REGISTRY

@DATASET_REGISTRY.register()
class PairedImageNetDataset(data.Dataset):
    
    def __init__(self, opt):
        super(PairedImageNetDataset, self).__init__()
        self.opt = opt
        self.file_client = None
        self.io_backend_opt = opt['io_backend']
        self.mean = opt['mean'] if 'mean' in opt else None
        self.std = opt['std'] if 'std' in opt else None
        self.gt_folder = opt['dataroot_gt']

        if self.io_backend_opt['type'] == 'lmdb':
            self.io_backend_opt['db_paths'] = [self.gt_folder]
            self.io_backend_opt['client_keys'] = ['gt']
            self.paths = paths_from_lmdb(self.gt_folder)
        elif 'meta_info_file' in self.opt:
            with open(self.opt['meta_info_file'], 'r') as fin:
     